In [1]:
import os
from PIL import Image
from torchvision import transforms

In [2]:
import torch
from tqdm import tqdm
from torchvision import models

In [3]:
# needed input dimensions for the CNN
inputDim = (224,224)
inputDir = 'dataset/'
inputDirCNN = 'dataset/'

In [4]:
os.makedirs(inputDirCNN, exist_ok = True)

In [5]:
transformationForCNNInput = transforms.Compose([transforms.Resize(inputDim)])

In [6]:
list_dir = os.listdir(inputDir)
list_dir.sort()

In [7]:
for imageName in list_dir:
    I = Image.open(os.path.join(inputDir, imageName)).convert('RGB')
    newI = transformationForCNNInput(I)

    exif = I.getexif()
    newI.save(os.path.join(inputDirCNN, imageName), exif=exif)

    newI.close()
    I.close()

In [8]:
class Img2VecResnet152():
    def __init__(self):

        self.device = torch.device("cpu")
        self.numberFeatures = 2048
        self.modelName = "resnet-152"
        self.model, self.featureLayer = self.getFeatureLayer()
        self.model = self.model.to(self.device)
        self.model.eval()
        self.toTensor = transforms.ToTensor()

        # normalize the resized images as expected by resnet152
        # [0.485, 0.456, 0.406] --> normalized mean value of ImageNet, [0.229, 0.224, 0.225] std of ImageNet
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    def getVec(self, img):
        image = self.normalize(self.toTensor(img)).unsqueeze(0).to(self.device)
        embedding = torch.zeros(1, self.numberFeatures, 1, 1)

        def copyData(m, i, o): embedding.copy_(o.data)

        h = self.featureLayer.register_forward_hook(copyData)
        self.model(image)
        h.remove()

        return embedding.numpy()[0, :, 0, 0]

    def getFeatureLayer(self):
        cnnModel = models.resnet152(pretrained=True)
        layer = cnnModel._modules.get('avgpool')
        # self.layer_output_size = 512
        self.layer_output_size = 2048

        return cnnModel, layer

In [9]:
# generate vectors for all the images in the set
img2vec = Img2VecResnet152()

allVectors = {}
print("Converting images to feature vectors:")
for image in tqdm(list_dir):
    I = Image.open(os.path.join(inputDirCNN, image))
    vec = img2vec.getVec(I)
    allVectors[image] = vec
    I.close()

/Users/fomin_ivan/miniconda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fomin_ivan/miniconda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Converting images to feature vectors:


100%|██████████| 9605/9605 [13:42<00:00, 11.68it/s]


In [10]:
# now let us define a function that calculates the cosine similarity entries in the similarity matrix
import pandas as pd
import numpy as np

def getSimilarityMatrix(vectors):
    v = np.array(list(vectors.values())).T
    sim = np.inner(v.T, v.T) / ((np.linalg.norm(v, axis=0).reshape(-1,1)) * ((np.linalg.norm(v, axis=0).reshape(-1,1)).T))
    keys = list(vectors.keys())
    matrix = pd.DataFrame(sim, columns = keys, index = keys)

    return matrix

similarityMatrix = getSimilarityMatrix(allVectors)

In [11]:
from numpy.testing import assert_almost_equal
import pickle

k = 7 # the number of top similar images to be stored

similarNames = pd.DataFrame(index = similarityMatrix.index, columns = range(k))

for j in tqdm(range(similarityMatrix.shape[0])):
    kSimilar = similarityMatrix.iloc[j, :].sort_values(ascending = False).head(k)
    similarNames.iloc[j, :] = list(kSimilar.index)

df = similarNames

100%|██████████| 9605/9605 [00:07<00:00, 1334.01it/s]


In [12]:
df.head()

,0,1,2,3,4,5,6
0000.png,0000.png,8222.png,2023.png,7420.png,7447.png,8492.png,1735.png
0001.png,0001.png,5193.png,5029.png,1995.png,1149.png,6470.png,3598.png
0002.png,0002.png,5492.png,7688.png,3977.png,1858.png,2448.png,5508.png
0003.png,0003.png,6017.png,1393.png,1348.png,5911.png,9169.png,3873.png
0004.png,0004.png,6371.png,4593.png,7238.png,0258.png,2470.png,2961.png


In [13]:
new_df = pd.DataFrame()
new_df['filename'] = df[0]
new_df['ranking'] = df[1] + ' ' + df[2] + ' ' + df[3] + ' ' + df[4] + ' ' + df[5] + ' ' + df[6]

In [14]:
new_df.head()

,filename,ranking
0000.png,0000.png,8222.png 2023.png 7420.png 7447.png 8492.png 1...
0001.png,0001.png,5193.png 5029.png 1995.png 1149.png 6470.png 3...
0002.png,0002.png,5492.png 7688.png 3977.png 1858.png 2448.png 5...
0003.png,0003.png,6017.png 1393.png 1348.png 5911.png 9169.png 3...
0004.png,0004.png,6371.png 4593.png 7238.png 0258.png 2470.png 2...


In [15]:
new_df.to_csv("BestSimilarNames152.csv", sep=',', index=False)

In [170]:
test = new_df['filename']
test = list(test)